In [49]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from scipy import stats
import warnings
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import matplotlib.pyplot as plt
import seaborn as sns
import re
import glob
!pip install xlsxwriter
pd.options.mode.chained_assignment = None  # default='warn'

In [16]:
def custom_sort(item):
    if item.endswith('basal'):
        return 0
    else:
        return int(item.split('uM')[0].split('_')[-1])

In [17]:
currdir= os.getcwd()
parent = os.path.dirname(currdir)
gparent=os.path.dirname(parent)
lines_to_skip = 10 # adjust

# count the number of header lines
header_lines = 3 # adjust
EXPERIMENT_TIME = "1_MIN"
plate_type = 'DIV19'

PERTURBATION = "CNO"
firstTableHeading = "Number of Spikes"

FOLDER_PATH = f"{parent}/data/Div19/LSL-Cas9-PVcre_5thbatch" # f"{parent}/data/DIV19_CNO/Div19_1minute" #
folder_dict = {}
balances= {"basal": "",
        "dose1": "",
        "dose2": "",
        "dose3": "",
        "dose4": ""
        }
## WALK Through folder of interest
for dirpath, dirnames, filenames in os.walk(FOLDER_PATH):
    if dirpath != FOLDER_PATH:
        folder_dict[dirpath.split("/")[-1]] = dirpath
    elif not dirnames:
        folder_dict['basal'] = dirpath

## get doses names in increasing order 
dose_names = [name for name in folder_dict.keys() if 'basal' not in name]
dose_names.sort(key=custom_sort)

# Initialize doses dictionaryd
balances = {"basal": "basal"}

## map sorted doses
for i, dose_name in enumerate(dose_names, start=1):
    balances[f"dose{i}"] = dose_name.split("_")[-1]

# Output the updated doses dictionary
print("Balances:", balances)
folder_dict

Balances: {'basal': 'basal'}


{'basal': '/home/poojaparameswaran/Documents/SoderlingLab/MultipleElectrodeAnalysisANOVA/data/Div19/LSL-Cas9-PVcre_5thbatch'}

In [18]:
def read_csvs_to_dict(folder, lines_to_skip=10):
    files = glob.glob(os.path.join(folder, "*.csv"))
    data_dict = {}
    for file in files:
        # Extracting the plate number from the filename
        plate_num = re.search(r'plate\s?(\d+)', file, re.IGNORECASE)
        if plate_num:
            plate_key = f"plate_{plate_num.group(1)}"
            df = pd.read_csv(file, skiprows = lines_to_skip)
            df = df.drop(columns=['Unnamed: 9'])
            data_dict[plate_key] = df
    return data_dict
all_files_dict = {}

for dose, folder in folder_dict.items():
    all_files_dict[dose] = read_csvs_to_dict(folder)
    
for dose, files_dict in all_files_dict.items():
    print(dose, files_dict.keys())

basal dict_keys(['plate_2', 'plate_3', 'plate_1'])


In [19]:
balances

{'basal': 'basal'}

In [20]:
def concat_plates(plates_dict, firstHeading):
    experiment = {}
    testparams_concat = {}
    for plate, df in plates_dict.items():
        maxrows = df.shape[0]
        vals_between_tables = 16
        titles = [heading_ind for heading_ind in range(14, maxrows+16, 16)]
        titles.insert(0,0)
        # # Empty dictionary to store dataframes
        # Loop through start indices
        for ind in range(len(titles) -1):
            if ind == 0:
                ## Handle mean firing rate
                key = firstHeading
                table = df.iloc[titles[ind] : titles[ind +1], :]
                # table = rename_row_names(table)
            elif ind != 0:
                table = df.iloc[titles[ind] : titles[ind +1], :]
                key = table.iloc[0][0] # just get name
                table = df.iloc[titles[ind] +2: titles[ind +1], :]

            table.set_index('Unnamed: 0', inplace=True)
            if key not in experiment:
                experiment[key] = []
            experiment[key].append(table)
    
    for testparam, df_list in experiment.items():
        concated_table = pd.concat(df_list, ignore_index=False)
        testparams_concat[testparam] = concated_table
    
    testparams_concat = {key.strip(): value for key, value in testparams_concat.items()}
    return testparams_concat
doses = {}
for dose, plates in all_files_dict.items():
    doses[dose] = concat_plates(plates, firstTableHeading)

doses.keys()

dict_keys(['basal'])

In [21]:
def remove_mean_SEM(test_dict, chemical_type):
    for key, value in test_dict.items():
        to_drop = [index for index in ['B Mean', 'B SEM'] if index in value.index]
        if to_drop:
            test_dict[key] = value.drop(index=to_drop)
        test_dict[key]['Chemical_Type']=chemical_type
    return test_dict
for dose, parameter_dict in doses.items():
    doses[dose] = remove_mean_SEM(parameter_dict, dose)


In [22]:
def rename_row_names(experiment_df):
    # 'Unnamed: 0' is a column
    experiment_df = experiment_df.reset_index()
    experiment_df = experiment_df.drop(columns="Unnamed: 0", errors="ignore")
    experiment_df.index = ["Replicate{}".format(i+1) for i in range(len(experiment_df))]
    return experiment_df
for dose, test_dict in doses.items():
    for test_name, df in test_dict.items():
        test_dict[test_name] = rename_row_names(df)
    doses[dose] = test_dict

In [23]:
doses['basal']['Number of Spikes']

,GPR37L 1,GPR37L 2,LGI2 A12,SLITRK5,THSD7 1,THSD7 2,unt1,unt2,Chemical_Type
Replicate1,0.103218,0.138529,0.057250,0.001254,0.343920,0.054952,0.001254,0.152110,basal
Replicate2,0.068115,0.033640,0.084413,0.060384,0.001463,0.055579,0.003761,0.000209,basal
Replicate3,0.199749,0.000209,0.012328,0.083159,0.006268,0.034476,0.024655,0.175930,basal
Replicate4,0.054325,0.203301,0.000000,0.311743,0.001672,0.050146,0.000418,0.001463,basal
Replicate5,0.041789,0.049728,0.045341,0.243209,0.001254,0.024028,0.052863,0.148140,basal
Replicate6,0.000000,0.166318,0.000418,0.063310,0.000627,0.040744,0.000000,0.002089,basal
Replicate7,0.000418,0.054325,0.000418,0.002089,0.000000,0.000000,0.029670,0.146887,basal
Replicate8,0.053280,0.061220,0.052445,0.255537,0.000418,0.033222,0.000418,0.136231,basal
Replicate9,0.043251,0.146051,0.000000,0.083159,0.000000,0.015880,0.002089,0.015671,basal
Replicate10,0.000418,0.077309,0.049519,0.316130,0.000000,0.067071,0.057459,0.117008,basal


In [24]:
balances.keys(), balances.values(), doses.keys()

(dict_keys(['basal']), dict_values(['basal']), dict_keys(['basal']))

In [28]:
dmapped = {}
for bkey, bval in balances.items():
    for dkey, dval in doses.items():
        if bval in dkey:
            dmapped[bkey] = dval

dmapped.keys()

dict_keys(['basal'])

In [52]:
for dose, tdict in dmapped.items():
    print(dose)

basal


In [58]:
def impute_nans(testparam, table):
    chem_types = table['Chemical_Type']
    table = table.drop(columns=['Chemical_Type'])
    original_index = table.index
    imputer = IterativeImputer(max_iter=100, random_state=0)
    imputed_data = imputer.fit_transform(table)

    # Convert the imputed data back to a DataFrame
    imputed_df = pd.DataFrame(imputed_data, columns=table.columns, index=original_index)
    imputed_df['Chemical_Type'] = chem_types
    return imputed_df
dmappedI = {}
for dose, tdict in dmapped.items():
    dmappedI[dose] = {testname: impute_nans(testname, table) for testname, table in tdict.items() if not table.loc[:, table.columns != 'Chemical_Type'].isna().all().all()} 


In [62]:
def perform_t_test(condition_dict):
    tests = {}
    for testname in condition_dict.keys():
        tests[testname] = {}
        table = condition_dict[testname].copy()  # Create a deep copy
        rows_to_drop = [row for row in ['B Mean', 'B SEM', 'B Std'] if any(row.lower() == existing_row.lower() for existing_row in table.index)]
        if rows_to_drop:
            table.drop(rows_to_drop, axis=0, inplace=True)
        test_variables = [col for col in table.columns if "unt" not in col.lower() and "chemical" not in col.lower()]
        control_name = "unt2"
        for testitem in test_variables:
            exp_group = list(table[testitem].astype(float))
            control_group = list(table[control_name].astype(float))
            with warnings.catch_warnings(record=True) as w:
                warnings.simplefilter("always")  # Cause all warnings to always be triggered
                t_stat, pval = stats.ttest_ind(exp_group, control_group)
                if len(w) > 0:
                    tests[testname][testitem] = [t_stat, pval, 'Warning: ' + str(w[-1].message)]
                else:
                    tests[testname][testitem] = [t_stat, pval]
    return tests

dmapped_ttest = {}
for key, testdict in dmappedI.items():
    dmapped_ttest[key] = perform_t_test(testdict)
dmapped_ttest

{'basal': {'Number of Spikes': {'GPR37L 1': [-1.0511819560667552,
    0.29679045125776327],
   'GPR37L 2': [-0.009828251652812611, 0.9921862729017878],
   'LGI2 A12': [-1.6428708659073532, 0.10489459751882362],
   'SLITRK5': [-0.9973079769873875, 0.32205013001558347],
   'THSD7 1': [-1.9215409601286195, 0.05873550758373053],
   'THSD7 2': [-1.511387947353418, 0.1351911449906407]},
  'Number of Active Electrodes': {'GPR37L 1': [-1.436724056538273,
    0.15525105352050858],
   'GPR37L 2': [-1.278815323489144, 0.20518550714916528],
   'LGI2 A12': [-1.524922552707138, 0.13178402598941988],
   'SLITRK5': [-0.3355141438577303, 0.7382407716647257],
   'THSD7 1': [-2.337946740671529, 0.02225342307384756],
   'THSD7 2': [-1.832628501256459, 0.07111090013308265]},
  'Weighted Mean Firing Rate (Hz)': {'GPR37L 1': [-1.1887129531146972,
    0.238569731265254],
   'GPR37L 2': [-0.05014845726202534, 0.9601469062878087],
   'LGI2 A12': [-1.6808703852055, 0.09724536512098882],
   'SLITRK5': [-1.1696450

In [77]:
dmapped_ttest['basal']

{'Number of Spikes': {'GPR37L 1': [-1.0511819560667552, 0.29679045125776327],
  'GPR37L 2': [-0.009828251652812611, 0.9921862729017878],
  'LGI2 A12': [-1.6428708659073532, 0.10489459751882362],
  'SLITRK5': [-0.9973079769873875, 0.32205013001558347],
  'THSD7 1': [-1.9215409601286195, 0.05873550758373053],
  'THSD7 2': [-1.511387947353418, 0.1351911449906407]},
 'Number of Active Electrodes': {'GPR37L 1': [-1.436724056538273,
   0.15525105352050858],
  'GPR37L 2': [-1.278815323489144, 0.20518550714916528],
  'LGI2 A12': [-1.524922552707138, 0.13178402598941988],
  'SLITRK5': [-0.3355141438577303, 0.7382407716647257],
  'THSD7 1': [-2.337946740671529, 0.02225342307384756],
  'THSD7 2': [-1.832628501256459, 0.07111090013308265]},
 'Weighted Mean Firing Rate (Hz)': {'GPR37L 1': [-1.1887129531146972,
   0.238569731265254],
  'GPR37L 2': [-0.05014845726202534, 0.9601469062878087],
  'LGI2 A12': [-1.6808703852055, 0.09724536512098882],
  'SLITRK5': [-1.1696450929450781, 0.2461110468605158],

In [63]:
def ensure_dirs_exists(path):
    directory = os.path.dirname(path)  # Get the directory part of the file path
    if not os.path.exists(directory):
        os.makedirs(directory)
    return

In [67]:
FOLDER_PATH.split("/")[-1]

'LSL-Cas9-PVcre_5thbatch'

In [78]:
# exp1.iloc[62:79]
def analyze_test_results(experiment_res, outfile):
    col_names = ['Test Types', 'Gene', 'T-statistic', 'Regulation', 'P-value', 'Significance', 'Warnings']
    df = pd.DataFrame(columns = col_names)

    ensure_dirs_exists(outfile)
    for test in (experiment_res.keys()):
        for prot in experiment_res[test].keys():
            new_row = {}
            new_row['Test Types'] = test
            new_row['Gene'] = prot
            t_stat = experiment_res[test][prot][0]
            new_row['T-statistic'] = t_stat
            if t_stat < 0:
                new_row['Regulation'] = "Down-Regulated"
            elif t_stat > 0:
                new_row['Regulation'] = "Up-Regulated"
            else:
                new_row['Regulation'] = "Same/NA"
            
            pval = experiment_res[test][prot][1]
            new_row['P-value'] = pval
            if pval <= 0.05:
                new_row['Significance'] = "Significant diff between gene and control"
            else:
                new_row['Significance'] = "Not Significant"
            if len(experiment_res[test][prot]) > 2:
                new_row['Warnings'] = experiment_res[test][prot][2]
            else:
                new_row['Warnings'] = "No Warning"

            df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
    ensure_dirs_exists(outfile)
    df.to_csv(outfile)
    return df

for condition, testdict in dmapped_ttest.items():
    test_results = analyze_test_results(testdict, 
                                        f'{parent}/t-tests_results/{FOLDER_PATH.split("/")[-2]}/{FOLDER_PATH.split("/")[-1]}/'
                                            f'{condition}_ttest_results.csv')
    test_results

In [89]:
def save_all_to_excel(data, outfile):
    ensure_dirs_exists(outfile)
    with pd.ExcelWriter(outfile, engine='xlsxwriter') as writer:
        for condition, testdict in data.items():
            # Flatten the data structure
            flat_data = {}
            for test, values in testdict.items():
                index = values.keys()
                flat_data[(test, 't-stat')] = [v[0] for v in values.values()]
                flat_data[(test, 'p-value')] = [v[1] for v in values.values()]

            # Create DataFrame with MultiIndex columns
            df = pd.DataFrame(flat_data, index =index)            
            df.columns = pd.MultiIndex.from_tuples(df.columns)

            df.to_excel(writer, sheet_name=condition)
    writer.close()
    if os.path.exists(outfile):
        return f"Outfile created {outfile}"
    else:
        return f"Problem in writing conditions to {outfile}"

save_all_to_excel(dmapped_ttest,f'{parent}/t-test_results/{FOLDER_PATH.split("/")[-2]}/{FOLDER_PATH.split("/")[-1]}/'
                                            f'{condition}_ttest_results.csv')


/home/poojaparameswaran/anaconda3/envs/dataenv/lib/python3.11/site-packages/xlsxwriter/workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


'File Created'

In [90]:

def save_all_to_excel(dataframes, outfile):
    # Convert single dataframe to a list of one dataframe for consistency
    if isinstance(dataframes, pd.DataFrame):
        dataframes = [dataframes]
        sheet_names = ['experiment 1']
    else:
        sheet_names = ['experiment {}'.format(i+1) for i in range(len(dataframes))]

    directory = os.path.dirname(outfile)
    if not os.path.exists(directory):
        os.makedirs(directory)
    writer = pd.ExcelWriter(outfile, engine='xlsxwriter')

    for df, sheetname in zip(dataframes, sheet_names):
        df.to_excel(writer, sheet_name=sheetname)
    
    writer.close()
    if os.path.exists(outfile):
        return f"Outfile created {outfile}"
    else:
        return "Problem in experiments to excel file"

save_all_to_excel(test_results, 
                  f'{parent}/t-test_results/{FOLDER_PATH.split("/")[-2]}/{FOLDER_PATH.split("/")[-1]}/'
                                            f'ttest_results.xlsx')

'File Created'